<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/output_parsing/GuardrailsDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guardrails Output Parsing


First, set your openai api keys

In [ ]:
# import os

# os.environ["OPENAI_API_KEY"] = "sk-..."

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
%pip install llama-index-llms-openai
%pip install llama-index-output-parsers-guardrails


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for guardrails-ai<0.4.0,>=0.3.2 from https://files.pythonhosted.org/packages/ae/5f/656bb84c9be700fff22719ddd1cb98b078dfbff51738f7e19998f4cf4d64/guardrails_ai-0.3.3-py3-none-any.whl.metadata
  Using cached guardrails_ai-0.3.3-py3-none-any.whl.metadata (8.6 kB)
Using cached guardrails_ai-0.3.3-py3-none-any.whl (148 kB)
  Attempting uninstall: guardrails-ai
    Found existing installation: guardrails-ai 0.4.2
    Uninstalling guardrails-ai-0.4.2:
      Successfully uninstalled guardrails-ai-0.4.2

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install guardrails-ai


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### Download Data

In [ ]:
!mkdir -p 'data/paul_graham/'
!curl 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' > 'data/paul_graham/paul_graham_essay.txt'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 75042  100 75042    0     0   390k      0 --:--:-- --:--:-- --:--:--  396k


#### Load documents, build the VectorStoreIndex


In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [ ]:
# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

In [ ]:
index = VectorStoreIndex.from_documents(documents, chunk_size=512)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


#### Define Query + Guardrails Spec


In [ ]:
from llama_index.output_parsers.guardrails import GuardrailsOutputParser

**Define custom QA and Refine Prompts**


**Define Guardrails Spec**


In [ ]:
# You can either define a RailSpec and initialise a Guard object from_rail_string()
# OR define Pydantic classes and initialise a Guard object from_pydantic()
# For more info: https://docs.guardrailsai.com/defining_guards/pydantic/
# Guardrails recommends Pydantic

from pydantic import BaseModel, Field
from typing import List
import guardrails as gd


class BulletPoints(BaseModel):
    # In all the fields below, you can define validators as well
    # Left out for brevity
    explanation: str = Field()
    explanation2: str = Field()
    explanation3: str = Field()


class Explanation(BaseModel):
    points: BulletPoints = Field(
        description="Bullet points regarding events in the author's life."
    )


# Define the prompt
prompt = """
Query string here.

${gr.xml_prefix_prompt}

${output_schema}

${gr.json_suffix_prompt_v2_wo_none}
"""

In [ ]:
from llama_index.llms.openai import OpenAI

# Create a guard object
guard = gd.Guard.from_pydantic(output_class=Explanation, prompt=prompt)

# Create output parse object
output_parser = GuardrailsOutputParser(guard)

# attach to an llm object
llm = OpenAI(output_parser=output_parser)

In [ ]:
from llama_index.core.prompts.default_prompts import (
    DEFAULT_TEXT_QA_PROMPT_TMPL,
)

# take a look at the new QA template!
fmt_qa_tmpl = output_parser.format(DEFAULT_TEXT_QA_PROMPT_TMPL)
print(fmt_qa_tmpl)

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


Given below is XML that describes the information to extract from this document and the tags to extract it into.


<output>
    <object name="points" description="Bullet points regarding events in the author's life.">
        <string name="explanation"/>
        <string name="explanation2"/>
        <string name="explanation3"/>
    </object>
</output>



ONLY return a valid JSON object (no other text is necessary). The JSON MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and specific types. Be correct and concise.




#### Query Index


In [ ]:
query_engine = index.as_query_engine(
    llm=llm,
)
response = query_engine.query(
    "What are the three items the author did growing up?",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [ ]:
print(response)

{'points': {'explanation': 'Writing short stories', 'explanation2': 'Programming on the IBM 1401 in 9th grade', 'explanation3': 'Getting a TRS-80 computer and starting programming in 1980'}}


In [ ]:
# View a summary of what the guard did
guard.history.last.tree

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │ No prompt                                                                                               │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Message History ────────────────────────────────────────────╮ │
    │ │ No message history.                                                                                     │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ {                                                                                                       │ │
    │ │     "points": {                                                                                         │ │
    │ │         "explanation": "Writing short stories",                                                         │ │
    │ │         "explanation2": "Programming on the IBM 1401 in 9th grade",                                     │ │
    │ │         "explanation3": "Getting a TRS-80 computer and starting programming in 1980"                    │ │
    │ │     }                                                                                                   │ │
    │ │ }                                                                                                       │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
    │ │ {                                                                                                       │ │
    │ │     'points': {                                                                                         │ │
    │ │         'explanation': 'Writing short stories',                                                         │ │
    │ │         'explanation2': 'Programming on the IBM 1401 in 9th grade',                                     │ │
    │ │         'explanation3': 'Getting a TRS-80 computer and starting programming in 1980'                    │ │
    │ │     }                                                                                                   │ │
    │ │ }                                                                                                       │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯